<a href="https://colab.research.google.com/github/tayfundaldik/ML-Projects/blob/main/TayfunTF2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf

import pandas as pd

In [1]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

In [3]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
# Here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe

573/573 [==============================] - 0s 0us/step


In [9]:
train.head()
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [10]:
def inputFunc(features, labels, batch_size =256, training = True):
  dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))

  if training:
    dataset = dataset.shuffle(1000).repeat()

  return dataset.batch(batch_size)


In [11]:
feature_columns = []
for key in train.keys():
  feature_columns.append(tf.feature_column.numeric_column(key=key))

In [15]:
from re import VERBOSE
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    hidden_units=[30,10],
    n_classes=3
)

classifier.train(input_fn=lambda: inputFunc(train,train_y,training=True),
                 steps=5000)

In [17]:
eval_result = classifier.evaluate(input_fn=lambda: inputFunc(test,test_y,training=False))
print('Test Accuracy: {accuracy:0.3f}\n'.format(**eval_result))

Test Accuracy: 0.767



In [19]:
def input_Fn(features, batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
  valid = True
  while valid:
    val = input(feature+": ")
    if not val.isdigit(): valid=False
  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_Fn(predict))
for pred_dictionary in predictions:
  class_id = pred_dictionary['class_ids'][0]
  probability = pred_dictionary['probabilities'][class_id]
  print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability))

Please type numeric values as prompted.
SepalLength: 3.0
SepalWidth: 5.4
PetalLength: 5.6
PetalWidth: 0.4
Prediction is "Setosa" (45.9%)
